# Setup

Note: the following code requires up to 30 GB of RAM over 2 hours if using 4 CPUs

## Import libraries for analysis

In [ ]:
import importlib
from glob import glob
from datetime import datetime

## Import calc_funcs module for use in analysis

In [ ]:
# Choose calc_funcs_ver to use in plotting scripts.
cfv = "latest"

assert (isinstance(cfv, str) & (len(cfv) == 5) & (cfv[:3] == "cfv") & 
        cfv[3].isnumeric() & cfv[4].isalpha() & cfv[4].islower()) | (cfv == "latest"), \
    ("cfv must be 'latest' or of form 'cfvXY' where X is a single digit number " +
     "and Y is a lowercase alphabet character. eg. cfv1n")

if cfv == "latest":
    calc_funcs_scripts = glob("calc_funcs_*.py")
    calc_funcs_scripts.sort()
    calc_funcs_module = calc_funcs_scripts[-1][:-3]
    
else:
    calc_funcs_module = "calc_funcs_" + cfv[2:]
    
cf = importlib.import_module(calc_funcs_module)

print(f"Using: {calc_funcs_module}")

## Import plot_funcs module for use in analysis

In [ ]:
# Choose plot_funcs_ver to use.
pfv = "latest"

assert (isinstance(pfv, str) & (len(pfv) == 5) & (pfv[:3] == "pfv") & 
        pfv[3].isnumeric() & pfv[4].isalpha() & pfv[4].islower()) | (pfv == "latest"), \
    ("pfv must be 'latest' or of form 'pfvXY' where X is a single digit number " +
     "and Y is a lowercase alphabet character. eg. pfv1e")

if pfv == "latest":
    plot_funcs_scripts = glob("plot_funcs_*.py")
    plot_funcs_scripts.sort()
    plot_funcs_module = plot_funcs_scripts[-1][:-3]
    
else:
    plot_funcs_module = "plot_funcs_" + pfv[2:]
    
pf = importlib.import_module(plot_funcs_module)

print(f"Using: {plot_funcs_module}")

## Analysis settings using selected periods from results below

Specify these all here in the beginning so errors from changes are minimised

In [ ]:
# General analysis settings
region = "wa"
year_start = 1983
year_end = 2019
window_size = 5
months_wet = "mam"
months_dry = "djf"

# Periods with similar background atmospheric oscillations
period1_mid_sim = "Dec-1999"
period1_start_sim = "Jun-1997"
period1_end_sim = "May-2002"
period2_mid_sim = "Mar-2013"
period2_start_sim = "Sep-2010"
period2_end_sim = "Aug-2015"
if (datetime.strptime(period1_start_sim, "%b-%Y") >= 
    datetime.strptime(cf.modis_earliest, "%b-%Y")):
    glass_source_pref_sim = "modis"
else:
    glass_source_pref_sim = "avhrr"

# Periods with different background atmospheric oscillations
period1_mid_dif = "Sep-1990"
period1_start_dif = "Mar-1988"
period1_end_dif = "Feb-1993"
period2_mid_dif = "Jun-2017"
period2_start_dif = "Dec-2014"
period2_end_dif = "Nov-2019"
if (datetime.strptime(period1_start_dif, "%b-%Y") >= 
    datetime.strptime(cf.modis_earliest, "%b-%Y")):
    glass_source_pref_dif = "modis"
else:
    glass_source_pref_dif = "avhrr"

# Single months with very different background atmospheric oscillations
month1_dif = "Nov-1996"
month2_dif = "Sep-2015"
if (datetime.strptime(month1_dif, "%b-%Y") >= 
    datetime.strptime(cf.modis_earliest, "%b-%Y")):
    glass_source_pref_dif_month = "modis"
else:
    glass_source_pref_dif_month = "avhrr"

# Selection of periods for analysis

## Years with extensive or concentrated leaf area index change

5-year rolling averages are used to average out LAI changes from short-term climate fluctuations

In [ ]:
# Create glass rolling avg files to identify years with significant LAI change
cf.calc_glass_rolling_avg_of_annual_diff(
    region=region, year_start=year_start, year_end=year_end, months_subset="all", 
    window_size=window_size, glass_source_pref="avhrr"
)
cf.calc_glass_rolling_avg_of_annual_diff(
    region=region, year_start=year_start, year_end=year_end, months_subset="all", 
    window_size=window_size, glass_source_pref="modis"
)

In [ ]:
# Plot the glass rolling avg files to identify years with significant LAI change 
# using AVHRR data
pf.create_glass_rolling_plot(
    region=region, year_start=year_start, year_end=year_end, months_subset="all", 
    window_size=window_size, param_glass_mean="mlai", glass_source_pref="avhrr", 
    extents=None, vmin=None, vmax=None, cfv_data=None, output=True
)

In [ ]:
# Plot the glass rolling avg files to identify years with significant LAI change 
# using MODIS data
pf.create_glass_rolling_plot(
    region=region, year_start=year_start, year_end=year_end, months_subset="all", 
    window_size=window_size, param_glass_mean="mlai", glass_source_pref="modis", 
    extents=None, vmin=None, vmax=None, cfv_data=None, output=True
)

In [ ]:
# Check the difference in MLAI for a selected set of periods
pf.create_individual_comp_plot(
    calc_func=cf.calc_glass_mean_clim, region=region, 
    period1_start=period1_start_sim, period1_end=period1_end_sim, 
    period2_start=period2_start_sim, period2_end=period2_end_sim, 
    months_subset="all", arg_extra="mlai", glass_source_pref=glass_source_pref_sim, 
    var_or_dvar=None, perc=False, mask_perc_quantile=pf.mask_perc_quantile_default, 
    mask_period1=None, mask_period2=None, extents=None, 
    vmin_periods=None, vmax_periods=None, vmin_diff=None, vmax_diff=None, 
    ax_period1=None, ax_period2=None, ax_diff=None, cfv_data=None, output=True
)

## Periods with similar / different background atmospheric oscillations

Select periods based on:
- similar/different pattern in monthly values over each period
- having similar/different 5-year rolling averages for relevant indices
- for WA this is mainly the AAOI and DMI
- for CA and SA this is mainly the AMOI, PDOI and ONI

In [ ]:
# Create climate indices plot to help select periods with similar background 
# atmospheric oscillations
pf.create_climate_indices_plot(
    year_start=year_start, year_end=year_end, window_size=window_size, 
    period1_mid="Dec-1999", period2_mid="Mar-2013", 
    month1_mark=None, month2_mark=None, 
    cfv_data=None, output=True
)

In [ ]:
# Create climate indices plot to help select periods with different background 
# atmospheric oscillations
pf.create_climate_indices_plot(
    year_start=year_start, year_end=year_end, window_size=window_size, 
    period1_mid=period1_mid_dif, period2_mid=period2_mid_dif, 
    month1_mark=month1_dif, month2_mark=month2_dif, 
    cfv_data=None, output=True
)

# Create all output files

## Create all output data files over each season

- this outputs all possible files for the difference in results between periods
- it does so by invoking the create_all_possible_diff_data_files function
- this function in turn calls upon the create_all_possible_calc_data_files function
- so intermediate data files for each period are also implicitly outputted

In [ ]:
# Create all possible diff data files for periods with similar background 
# atmospheric oscillations (year round, wet season only, and dry season only)
cf.create_all_possible_diff_data_files(
    region=region, period1_start=period1_start_sim, period1_end=period1_end_sim,
    period2_start=period2_start_sim, period2_end=period2_end_sim, months_subset="all"
)
cf.create_all_possible_diff_data_files(
    region=region, period1_start=period1_start_sim, period1_end=period1_end_sim,
    period2_start=period2_start_sim, period2_end=period2_end_sim, months_subset=months_wet
)
cf.create_all_possible_diff_data_files(
    region=region, period1_start=period1_start_sim, period1_end=period1_end_sim,
    period2_start=period2_start_sim, period2_end=period2_end_sim, months_subset=months_dry
)

In [ ]:
# Create all possible diff data files for periods with different background 
# atmospheric oscillations (year round, wet season only, and dry season only)
cf.create_all_possible_diff_data_files(
    region=region, period1_start=period1_start_dif, period1_end=period1_end_dif,
    period2_start=period2_start_dif, period2_end=period2_end_dif, months_subset="all"
)
cf.create_all_possible_diff_data_files(
    region=region, period1_start=period1_start_dif, period1_end=period1_end_dif,
    period2_start=period2_start_dif, period2_end=period2_end_dif, months_subset=months_wet
)
cf.create_all_possible_diff_data_files(
    region=region, period1_start=period1_start_dif, period1_end=period1_end_dif,
    period2_start=period2_start_dif, period2_end=period2_end_dif, months_subset=months_dry
)

In [ ]:
# Create all possible diff data files for months with different background 
# atmospheric oscillations
cf.create_all_possible_diff_data_files(
    region=region, period1_start=month1_dif, period1_end=month1_dif,
    period2_start=month2_dif, period2_end=month2_dif, months_subset="all"
)

## Create all static plot files

In [ ]:
# Create climate indices plot with no labels
pf.create_climate_indices_plot(
    year_start=year_start, year_end=year_end, window_size=window_size, 
    period1_mid=None, period2_mid=None, month1_mark=None, month2_mark=None, 
    cfv_data=None, output=True
)

In [ ]:
# Plot for land surface elevation
pf.create_orog_static_plot(
    param_orog="lse", region=region, extents=None, vmin=None, vmax=None, 
    ax=None, cfv_data=None, output=True
)

In [ ]:
# Plot for slope of sub-gridscale orography
pf.create_orog_static_plot(
    param_orog="ssgo", region=region, extents=None, vmin=None, vmax=None, 
    ax=None, cfv_data=None, output=True
)

## Create all output comparison plot files over each season

In [ ]:
# Create all possible comp plot files for periods with similar background 
# atmospheric oscillations (year round, wet season only, and dry season only)
pf.create_all_possible_comp_plot_files(
    region=region, period1_start=period1_start_sim, period1_end=period1_end_sim, 
    period2_start=period2_start_sim, period2_end=period2_end_sim, months_subset="all", 
    glass_source_pref=glass_source_pref_sim, perc=False, 
    mask_perc_quantile=pf.mask_perc_quantile_default, 
    mask_period1=None, mask_period2=None, extents=None, cfv_data=None
)
pf.create_all_possible_comp_plot_files(
    region=region, period1_start=period1_start_sim, period1_end=period1_end_sim, 
    period2_start=period2_start_sim, period2_end=period2_end_sim, months_subset=months_wet, 
    glass_source_pref=glass_source_pref_sim, perc=False, 
    mask_perc_quantile=pf.mask_perc_quantile_default, 
    mask_period1=None, mask_period2=None, extents=None, cfv_data=None
)
pf.create_all_possible_comp_plot_files(
    region=region, period1_start=period1_start_sim, period1_end=period1_end_sim, 
    period2_start=period2_start_sim, period2_end=period2_end_sim, months_subset=months_dry, 
    glass_source_pref=glass_source_pref_sim, perc=False, 
    mask_perc_quantile=pf.mask_perc_quantile_default, 
    mask_period1=None, mask_period2=None, extents=None, cfv_data=None
)

In [ ]:
# Create all possible comp plot files for periods with different background 
# atmospheric oscillations (year round, wet season only, and dry season only)
pf.create_all_possible_comp_plot_files(
    region=region, period1_start=period1_start_dif, period1_end=period1_end_dif, 
    period2_start=period2_start_dif, period2_end=period2_end_dif, months_subset="all", 
    glass_source_pref=glass_source_pref_dif, perc=False, 
    mask_perc_quantile=pf.mask_perc_quantile_default, 
    mask_period1=None, mask_period2=None, extents=None, cfv_data=None
)
pf.create_all_possible_comp_plot_files(
    region=region, period1_start=period1_start_dif, period1_end=period1_end_dif, 
    period2_start=period2_start_dif, period2_end=period2_end_dif, months_subset=months_wet, 
    glass_source_pref=glass_source_pref_dif, perc=False, 
    mask_perc_quantile=pf.mask_perc_quantile_default, 
    mask_period1=None, mask_period2=None, extents=None, cfv_data=None
)
pf.create_all_possible_comp_plot_files(
    region=region, period1_start=period1_start_dif, period1_end=period1_end_dif, 
    period2_start=period2_start_dif, period2_end=period2_end_dif, months_subset=months_dry, 
    glass_source_pref=glass_source_pref_dif, perc=False, 
    mask_perc_quantile=pf.mask_perc_quantile_default, 
    mask_period1=None, mask_period2=None, extents=None, cfv_data=None
)

In [ ]:
# Create all possible comp plot files for months with different background 
# atmospheric oscillations
pf.create_all_possible_comp_plot_files(
    region=region, period1_start=month1_dif, period1_end=month1_dif, 
    period2_start=month2_dif, period2_end=month2_dif, months_subset="all", 
    glass_source_pref=glass_source_pref_dif_month, perc=False, 
    mask_perc_quantile=pf.mask_perc_quantile_default, 
    mask_period1=None, mask_period2=None, extents=None, cfv_data=None
)

# Analysis

In [ ]:
pf.plot_comp_mdp_clim_stats_given_var_or_dvar(
    region=region, period1_start=period1_start_sim, period1_end=period1_end_sim, 
    period2_start=period2_start_sim, period2_end=period2_end_sim, months_subset="all", 
    glass_source_pref=glass_source_pref_sim, var_or_dvar="wv100", 
    perc=False, mask_perc_quantile=pf.mask_perc_quantile_default, 
    mask_period1=None, mask_period2=None, extents=None, cfv_data=None, output=True
)

In [ ]:
pf.plot_comp_mdp_clim_stats_given_var_or_dvar(
    region=region, period1_start=period1_start_sim, period1_end=period1_end_sim, 
    period2_start=period2_start_sim, period2_end=period2_end_sim, months_subset="all", 
    glass_source_pref=glass_source_pref_sim, var_or_dvar="ws100", 
    perc=False, mask_perc_quantile=pf.mask_perc_quantile_default, 
    mask_period1=None, mask_period2=None, extents=None, cfv_data=None, output=True
)

In [ ]:
pf.plot_comp_mdp_clim_stats_given_var_or_dvar(
    region=region, period1_start=period1_start_sim, period1_end=period1_end_sim, 
    period2_start=period2_start_sim, period2_end=period2_end_sim, months_subset="all", 
    glass_source_pref=glass_source_pref_sim, var_or_dvar="u100", 
    perc=False, mask_perc_quantile=pf.mask_perc_quantile_default, 
    mask_period1=None, mask_period2=None, extents=None, cfv_data=None, output=True
)

In [ ]:
pf.plot_comp_mdp_clim_stats_given_var_or_dvar(
    region=region, period1_start=period1_start_sim, period1_end=period1_end_sim, 
    period2_start=period2_start_sim, period2_end=period2_end_sim, months_subset="all", 
    glass_source_pref=glass_source_pref_sim, var_or_dvar="v100", 
    perc=False, mask_perc_quantile=pf.mask_perc_quantile_default, 
    mask_period1=None, mask_period2=None, extents=None, cfv_data=None, output=True
)

In [ ]:
pf.plot_comp_mdp_clim_stats_given_var_or_dvar(
    region=region, period1_start=period1_start_sim, period1_end=period1_end_sim, 
    period2_start=period2_start_sim, period2_end=period2_end_sim, months_subset="all", 
    glass_source_pref=glass_source_pref_sim, var_or_dvar="mslp", 
    perc=False, mask_perc_quantile=pf.mask_perc_quantile_default, 
    mask_period1=None, mask_period2=None, extents=None, cfv_data=None, output=True
)

In [ ]:
pf.plot_comp_mdp_clim_stats_given_var_or_dvar(
    region=region, period1_start=period1_start_sim, period1_end=period1_end_sim, 
    period2_start=period2_start_sim, period2_end=period2_end_sim, months_subset="all", 
    glass_source_pref=glass_source_pref_sim, var_or_dvar="t2", 
    perc=False, mask_perc_quantile=pf.mask_perc_quantile_default, 
    mask_period1=None, mask_period2=None, extents=None, cfv_data=None, output=True
)

In [ ]:
pf.plot_comp_mdp_clim_stats_given_var_or_dvar(
    region=region, period1_start=period1_start_sim, period1_end=period1_end_sim, 
    period2_start=period2_start_sim, period2_end=period2_end_sim, months_subset="all", 
    glass_source_pref=glass_source_pref_sim, var_or_dvar="nac", 
    perc=False, mask_perc_quantile=pf.mask_perc_quantile_default, 
    mask_period1=None, mask_period2=None, extents=None, cfv_data=None, output=True
)

# Scrap

# Comparison 1

- period1 = Jan-1994 to Dec-1998
- period2 = Jan-2004 to Dec-2008
- extents = [113, 123, -35, -30]
- tz = UTC +8
- glass_data_source = AVHRR

## Comparison 1, year round (all months)

### Comparison 1, year round (all months), MDP climatology stats

In [ ]:
pf.plot_comp_mdp_clim_stats_given_var_or_dvar(
    region="wa", period1_start="Jan-1994", period1_end="Dec-1998", 
    period2_start="Jan-2004", period2_end="Dec-2008", months_subset="all", 
    glass_source_pref="avhrr", var_or_dvar="mslp", perc=False, 
    mask_perc_quantile=10, mask_period1=None, 
    mask_period2=None, extents=None, cfv_data=None, output=True
)

In [ ]:
pf.plot_comp_mdp_clim_stats_given_var_or_dvar(
    region="wa", period1_start="Jan-1994", period1_end="Dec-1998", 
    period2_start="Jan-2004", period2_end="Dec-2008", months_subset="all", 
    glass_source_pref="avhrr", var_or_dvar="nac", perc=False, 
    mask_perc_quantile=20, mask_period1=None, 
    mask_period2=None, extents=None, cfv_data=None, output=True
)

In [ ]:
pf.plot_comp_mdp_clim_stats_given_var_or_dvar(
    region="wa", period1_start="Jan-1994", period1_end="Dec-1998", 
    period2_start="Jan-2004", period2_end="Dec-2008", months_subset="all", 
    glass_source_pref="avhrr", var_or_dvar="u100", perc=False, 
    mask_perc_quantile=20, mask_period1=None, 
    mask_period2=None, extents=None, cfv_data=None, output=True
)

In [ ]:
pf.plot_comp_mdp_clim_stats_given_var_or_dvar(
    region="wa", period1_start="Jan-1994", period1_end="Dec-1998", 
    period2_start="Jan-2004", period2_end="Dec-2008", months_subset="all", 
    glass_source_pref="avhrr", var_or_dvar="v100", perc=False, 
    mask_perc_quantile=20, mask_period1=None, 
    mask_period2=None, extents=None, cfv_data=None, output=True
)

In [ ]:
pf.plot_comp_mdp_clim_stats_given_var_or_dvar(
    region="wa", period1_start="Jan-1994", period1_end="Dec-1998", 
    period2_start="Jan-2004", period2_end="Dec-2008", months_subset="all", 
    glass_source_pref="avhrr", var_or_dvar="t2", perc=False, 
    mask_perc_quantile=20, mask_period1=None, 
    mask_period2=None, extents=None, cfv_data=None, output=True
)

In [ ]:
pf.plot_comp_mdp_clim_stats_given_var_or_dvar(
    region="wa", period1_start="Jan-1994", period1_end="Dec-1998", 
    period2_start="Jan-2004", period2_end="Dec-2008", months_subset="all", 
    glass_source_pref="avhrr", var_or_dvar="ws100", perc=False, 
    mask_perc_quantile=10, mask_period1=None, 
    mask_period2=None, extents=None, cfv_data=None, output=True
)

In [ ]:
pf.plot_comp_mdp_clim_stats_given_var_or_dvar(
    region="wa", period1_start="Jan-1994", period1_end="Dec-1998", 
    period2_start="Jan-2004", period2_end="Dec-2008", months_subset="all", 
    glass_source_pref="avhrr", var_or_dvar="wv100", perc=False, 
    mask_perc_quantile=10, mask_period1=None, 
    mask_period2=None, extents=None, cfv_data=None, output=True
)

In [ ]:
pf.plot_comp_mdp_clim_stats_given_var_or_dvar(
    region="wa", period1_start="Jan-1994", period1_end="Dec-1998", 
    period2_start="Jan-2004", period2_end="Dec-2008", months_subset="all", 
    glass_source_pref="avhrr", var_or_dvar="slhf", perc=False, 
    mask_perc_quantile=10, mask_period1=None, 
    mask_period2=None, extents=None, cfv_data=None, output=True
)

In [ ]:
pf.plot_comp_mdp_clim_stats_given_var_or_dvar(
    region="wa", period1_start="Jan-1994", period1_end="Dec-1998", 
    period2_start="Jan-2004", period2_end="Dec-2008", months_subset="all", 
    glass_source_pref="avhrr", var_or_dvar="sshf", perc=False, 
    mask_perc_quantile=10, mask_period1=None, 
    mask_period2=None, extents=None, cfv_data=None, output=True
)

In [ ]:
pf.plot_comp_mdp_clim_stats_given_var_or_dvar(
    region="wa", period1_start="Jan-1994", period1_end="Dec-1998", 
    period2_start="Jan-2004", period2_end="Dec-2008", months_subset="all", 
    glass_source_pref="avhrr", var_or_dvar="nse", perc=False, 
    mask_perc_quantile=10, mask_period1=None, 
    mask_period2=None, extents=None, cfv_data=None, output=True
)

In [ ]:
pf.plot_comp_mdp_clim_stats_given_var_or_dvar(
    region="wa", period1_start="Jan-1994", period1_end="Dec-1998", 
    period2_start="Jan-2004", period2_end="Dec-2008", months_subset="all", 
    glass_source_pref="avhrr", var_or_dvar="vidmf", perc=False, 
    mask_perc_quantile=10, mask_period1=None, 
    mask_period2=None, extents=None, cfv_data=None, output=True
)

In [ ]:
pf.plot_comp_mdp_clim_stats_given_var_or_dvar(
    region="wa", period1_start="Jan-1994", period1_end="Dec-1998", 
    period2_start="Jan-2004", period2_end="Dec-2008", months_subset="all", 
    glass_source_pref="avhrr", var_or_dvar="viec", perc=False, 
    mask_perc_quantile=10, mask_period1=None, 
    mask_period2=None, extents=None, cfv_data=None, output=True
)

In [ ]:
pf.plot_comp_mdp_clim_stats_given_var_or_dvar(
    region="wa", period1_start="Jan-1994", period1_end="Dec-1998", 
    period2_start="Jan-2004", period2_end="Dec-2008", months_subset="all", 
    glass_source_pref="avhrr", var_or_dvar="vipile", perc=False, 
    mask_perc_quantile=10, mask_period1=None, 
    mask_period2=None, extents=None, cfv_data=None, output=True
)

In [ ]:
pf.plot_comp_mdp_clim_stats_given_var_or_dvar(
    region="wa", period1_start="Jan-1994", period1_end="Dec-1998", 
    period2_start="Jan-2004", period2_end="Dec-2008", months_subset="all", 
    glass_source_pref="avhrr", var_or_dvar="vike", perc=False, 
    mask_perc_quantile=10, mask_period1=None, 
    mask_period2=None, extents=None, cfv_data=None, output=True
)

In [ ]:
pf.plot_comp_mdp_clim_stats_given_var_or_dvar(
    region="wa", period1_start="Jan-1994", period1_end="Dec-1998", 
    period2_start="Jan-2004", period2_end="Dec-2008", months_subset="all", 
    glass_source_pref="avhrr", var_or_dvar="tcclw", perc=False, 
    mask_perc_quantile=10, mask_period1=None, 
    mask_period2=None, extents=None, cfv_data=None, output=True
)

In [ ]:
pf.plot_comp_mdp_clim_stats_given_var_or_dvar(
    region="wa", period1_start="Jan-1994", period1_end="Dec-1998", 
    period2_start="Jan-2004", period2_end="Dec-2008", months_subset="all", 
    glass_source_pref="avhrr", var_or_dvar="tcwv", perc=False, 
    mask_perc_quantile=10, mask_period1=None, 
    mask_period2=None, extents=None, cfv_data=None, output=True
)

In [ ]:
pf.plot_comp_mdp_clim_stats_given_var_or_dvar(
    region="wa", period1_start="Jan-1994", period1_end="Dec-1998", 
    period2_start="Jan-2004", period2_end="Dec-2008", months_subset="all", 
    glass_source_pref="avhrr", var_or_dvar="blh", perc=False, 
    mask_perc_quantile=10, mask_period1=None, 
    mask_period2=None, extents=None, cfv_data=None, output=True
)

In [ ]:
pf.plot_comp_mdp_clim_stats_given_var_or_dvar(
    region="wa", period1_start="Jan-1994", period1_end="Dec-1998", 
    period2_start="Jan-2004", period2_end="Dec-2008", months_subset="all", 
    glass_source_pref="avhrr", var_or_dvar="fa", perc=False, 
    mask_perc_quantile=10, mask_period1=None, 
    mask_period2=None, extents=None, cfv_data=None, output=True
)

In [ ]:
pf.plot_comp_mdp_clim_stats_given_var_or_dvar(
    region="wa", period1_start="Jan-1994", period1_end="Dec-1998", 
    period2_start="Jan-2004", period2_end="Dec-2008", months_subset="all", 
    glass_source_pref="avhrr", var_or_dvar="cbh", perc=False, 
    mask_perc_quantile=10, mask_period1=None, 
    mask_period2=None, extents=None, cfv_data=None, output=True
)

In [ ]:
pf.plot_comp_mdp_clim_stats_given_var_or_dvar(
    region="wa", period1_start="Jan-1994", period1_end="Dec-1998", 
    period2_start="Jan-2004", period2_end="Dec-2008", months_subset="all", 
    glass_source_pref="avhrr", var_or_dvar="tcc", perc=False, 
    mask_perc_quantile=10, mask_period1=None, 
    mask_period2=None, extents=None, cfv_data=None, output=True
)

In [ ]:
pf.plot_comp_mdp_clim_stats_given_var_or_dvar(
    region="wa", period1_start="Jan-1994", period1_end="Dec-1998", 
    period2_start="Jan-2004", period2_end="Dec-2008", months_subset="all", 
    glass_source_pref="avhrr", var_or_dvar="cape", perc=False, 
    mask_perc_quantile=10, mask_period1=None, 
    mask_period2=None, extents=None, cfv_data=None, output=True
)

In [ ]:
pf.plot_comp_mdp_clim_stats_given_var_or_dvar(
    region="wa", period1_start="Jan-1994", period1_end="Dec-1998", 
    period2_start="Jan-2004", period2_end="Dec-2008", months_subset="all", 
    glass_source_pref="avhrr", var_or_dvar="ci", perc=False, 
    mask_perc_quantile=10, mask_period1=None, 
    mask_period2=None, extents=None, cfv_data=None, output=True
)

### Comparison 1, year round (all months), MDP climatology values

### Comparison 1, year round (all months), WSD climatology

In [ ]:
pf.plot_comp_wsd_clim(
    region="wa", period1_start="Jan-1994", period1_end="Dec-1998", 
    period2_start="Jan-2004", period2_end="Dec-2008", months_subset="all",
    glass_source_pref="avhrr", perc=False, mask_perc_quantile=10, 
    mask_period1=None, mask_period2=None, extents=None, cfv_data=None, output=True
)

## Comparison 1, wet season (MAM months)

### Comparison 1, wet season (MAM months), MDP climatology stats

### Comparison 1, wet season (MAM months), MDP climatology values

### Comparison 1, wet season (MAM months), WSD climatology

## Comparison 1, dry season (DJF months)

### Comparison 1, dry season (DJF months), MDP climatology stats

### Comparison 1, dry season (DJF months), MDP climatology values

### Comparison 1, dry season (DJF months), WSD climatology

# Comparison 2

- period1 = Jan-2019 to Dec-2019
- period2 = Jan-2021 to Dec-2021
- extents = [113, 123, -35, -30] or [117, 122, -35, -30]
- tz = UTC +8
- glass_data_source = MODIS

## Comparison 2, year round (all months)

### Comparison 2, year round (all months), MDP climatology stats

### Comparison 2, year round (all months), MDP climatology values

### Comparison 2, year round (all months), WSD climatology

## Comparison 2, wet season (MAM months)

### Comparison 2, wet season (MAM months), MDP climatology stats

### Comparison 2, wet season (MAM months), MDP climatology values

### Comparison 2, wet season (MAM months), WSD climatology

## Comparison 2, dry season (DJF months)

### Comparison 2, dry season (DJF months), MDP climatology stats

### Comparison 2, dry season (DJF months), MDP climatology values

### Comparison 2, dry season (DJF months), WSD climatology